In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# -----------------------------
# Residual Block with 128 Filters
# -----------------------------
class ResidualBlock(nn.Module):
    def __init__(self, channels=128):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        return self.relu(out + residual)

# -----------------------------
# LSC_CNN Model
# -----------------------------
class LSC_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Initial convolution layer
        self.conv1 = nn.Conv2d(1, 128, kernel_size=7, padding=3)
        self.relu1 = nn.ReLU(inplace=True)

        # Intermediate convolution layers
        self.conv2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU(inplace=True)

        self.conv3 = nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1)
        self.relu3 = nn.ReLU(inplace=True)

        # Residual blocks with 128 channels
        self.res_blocks = nn.Sequential(*[ResidualBlock(128) for _ in range(20)])

        # Upsampling and final refinement layers
        self.conv21 = nn.Conv2d(1 + 128, 64, kernel_size=3, padding=1)
        self.relu21 = nn.ReLU(inplace=True)
        self.conv22 = nn.Conv2d(64, 1, kernel_size=3, padding=1)

    def forward(self, x):
        inp = x  # Save the original input

        # Encoding
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.relu3(self.conv3(x))

        # Residual feature extraction
        x = self.res_blocks(x)

        # Upsampling to match input size
        up = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)

        # Match size with original input
        h = min(inp.shape[2], up.shape[2])
        w = min(inp.shape[3], up.shape[3])
        inp = inp[:, :, :h, :w]
        up = up[:, :, :h, :w]

        # Element-wise multiplication and concatenation
        enhanced = up * inp
        concat = torch.cat((inp, enhanced), dim=1)

        # Refinement and noise prediction
        x = self.relu21(self.conv21(concat))
        noise = self.conv22(x)

        # Final clean image
        return inp - noise


In [5]:
# dataset.py
import os
from PIL import Image
from torchvision import transforms as T
from torch.utils.data import Dataset
import torch

class PairedIRDataset(Dataset):
    def __init__(self, clean_dir, noisy_dir, crop_size=50):
        self.clean_dir = clean_dir
        self.noisy_dir = noisy_dir
        self.filenames = sorted([
            f for f in os.listdir(clean_dir)
            if f.lower().endswith(('.jpg', '.png', '.bmp'))
        ])
        self.crop_size = crop_size
        self.transform = T.ToTensor()

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        fname = self.filenames[idx]
        clean = Image.open(os.path.join(self.clean_dir, fname)).convert("L")
        noisy = Image.open(os.path.join(self.noisy_dir, fname.replace("Original", ""))).convert("L")

        clean = self.transform(clean)
        noisy = self.transform(noisy)

        _, h, w = clean.shape
        top = torch.randint(0, h - self.crop_size + 1, (1,)).item()
        left = torch.randint(0, w - self.crop_size + 1, (1,)).item()
        clean = clean[:, top:top+self.crop_size, left:left+self.crop_size]
        noisy = noisy[:, top:top+self.crop_size, left:left+self.crop_size]

        return noisy, clean


In [3]:
# train.py

import os
import torch
from torch.utils.data import DataLoader
from torch import nn
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
#from model import LSC_CNN
#from dataset import PairedIRDataset

def compute_metrics(pred, target):
    pred_np = pred.cpu().numpy().squeeze()
    target_np = target.cpu().numpy().squeeze()
    psnr_vals, ssim_vals = [], []
    for i in range(pred_np.shape[0]):
        psnr_vals.append(peak_signal_noise_ratio(target_np[i], pred_np[i], data_range=1.0))
        ssim_vals.append(structural_similarity(target_np[i], pred_np[i], data_range=1.0))
    return np.mean(psnr_vals), np.mean(ssim_vals)

# -----------------------------
# ✅ Updated Paths
# -----------------------------
train_clean_dir = "/home/himanshu/Desktop/NUC_DATA/newfirstoriginal"
train_noisy_dir = "/home/himanshu/Desktop/NUC_DATA/sigmareduced"
val_clean_dir   = "/home/himanshu/Desktop/NUC_DATA/newfirstoriginal"
val_noisy_dir   = "/home/himanshu/Desktop/NUC_DATA/newfirstnoise"
checkpoint_dir  = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Hyperparameters
batch_size = 10      # 16
num_epochs = 25
lr = 1e-3
crop_size = 50

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSC_CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.MSELoss()

# Load data
train_dataset = PairedIRDataset(train_clean_dir, train_noisy_dir, crop_size)
val_dataset   = PairedIRDataset(val_clean_dir, val_noisy_dir, crop_size)
train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader    = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Training
best_loss = float("inf")
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for noisy, clean in train_loader:
        noisy, clean = noisy.to(device), clean.to(device)
        pred = model(noisy)
        loss = loss_fn(pred, clean)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * noisy.size(0)
    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss, psnr_total, ssim_total = 0.0, 0.0, 0.0
    with torch.no_grad():
        for noisy, clean in val_loader:
            noisy, clean = noisy.to(device), clean.to(device)
            pred = model(noisy)
            val_loss += loss_fn(pred, clean).item() * noisy.size(0)
            psnr, ssim = compute_metrics(pred, clean)
            psnr_total += psnr * noisy.size(0)
            ssim_total += ssim * noisy.size(0)

    val_loss /= len(val_loader.dataset)
    psnr_total /= len(val_loader.dataset)
    ssim_total /= len(val_loader.dataset)

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), f"{checkpoint_dir}/best_modelone.pth")
        print(f"✅ Best model saved at epoch {epoch+1} — Val Loss: {val_loss:.6f}")

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | PSNR: {psnr_total:.2f} | SSIM: {ssim_total:.4f}")

torch.save(model.state_dict(), f"{checkpoint_dir}/final_modelone.pth")
print(f"✅ Final model saved to {checkpoint_dir}/final_modelone.pth")


/home/himanshu/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/pytorch/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/himanshu/.local/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/pytorch/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


✅ Best model saved at epoch 1 — Val Loss: 0.002233
Epoch 1/25 | Train Loss: 0.052296 | Val Loss: 0.002233 | PSNR: 26.70 | SSIM: 0.6183
✅ Best model saved at epoch 2 — Val Loss: 0.001578
Epoch 2/25 | Train Loss: 0.001871 | Val Loss: 0.001578 | PSNR: 28.27 | SSIM: 0.7211
✅ Best model saved at epoch 3 — Val Loss: 0.001461
Epoch 3/25 | Train Loss: 0.001578 | Val Loss: 0.001461 | PSNR: 28.71 | SSIM: 0.7502
Epoch 4/25 | Train Loss: 0.001423 | Val Loss: 0.001814 | PSNR: 28.18 | SSIM: 0.7383
✅ Best model saved at epoch 5 — Val Loss: 0.001328
Epoch 5/25 | Train Loss: 0.001349 | Val Loss: 0.001328 | PSNR: 29.26 | SSIM: 0.7912
✅ Best model saved at epoch 6 — Val Loss: 0.001212
Epoch 6/25 | Train Loss: 0.001286 | Val Loss: 0.001212 | PSNR: 29.55 | SSIM: 0.7927
✅ Best model saved at epoch 7 — Val Loss: 0.001125
Epoch 7/25 | Train Loss: 0.001246 | Val Loss: 0.001125 | PSNR: 30.01 | SSIM: 0.8209
✅ Best model saved at epoch 8 — Val Loss: 0.001100
Epoch 8/25 | Train Loss: 0.001214 | Val Loss: 0.001100 

In [16]:
import os
import cv2
import torch
import numpy as np
from torchvision import transforms as T
from PIL import Image
from model import LSC_CNN  # Ensure your model is defined correctly
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

# ----------------------------- #
# Configuration
# ----------------------------- #
input_path = ""  # Change as needed
output_dir = "/home/himanshu/Desktop/NUC_DATA/nuc_dataset/dataset/test_data"
model_path = "checkpoints/final_modelone.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(output_dir, exist_ok=True)

# ----------------------------- #
# Load Model
# ----------------------------- #
model = LSC_CNN().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# ----------------------------- #
# Transforms
# ----------------------------- #
transform = T.ToTensor()
to_pil = T.ToPILImage()

# ----------------------------- #
# Helper: RNS Calculation
# ----------------------------- #
def calculate_rns(tensor_img):
    mean_val = torch.mean(tensor_img)
    std_val = torch.std(tensor_img)
    return (std_val / (mean_val + 1e-8)).item()

# ----------------------------- #
# PSNR and SSIM Calculation
# ----------------------------- #
def calculate_psnr_ssim(img1, img2):
    img1_np = np.array(img1, dtype=np.float32) / 255.0
    img2_np = np.array(img2, dtype=np.float32) / 255.0
    psnr = compare_psnr(img1_np, img2_np, data_range=1.0)
    ssim = compare_ssim(img1_np, img2_np, data_range=1.0)
    return psnr, ssim

# ----------------------------- #
# Denoising Inference
# ----------------------------- #
def denoise_image(img: Image.Image):
    img_gray = img.convert("L")
    img_tensor = transform(img_gray).unsqueeze(0).to(device)
    with torch.no_grad():
        output_tensor = model(img_tensor)
    input_tensor = img_tensor.squeeze().cpu().clamp(0, 1)
    output_tensor = output_tensor.squeeze().cpu().clamp(0, 1)
    return input_tensor, output_tensor

# ----------------------------- #
# Process Single Image
# ----------------------------- #
def process_single_image(image_path):
    img = Image.open(image_path).convert("L")
    input_tensor, output_tensor = denoise_image(img)

    # Convert to PIL for metric calculation
    input_img_pil = to_pil(input_tensor)
    output_img_pil = to_pil(output_tensor)

    # RNS
    rns_input = calculate_rns(input_tensor)
    rns_output = calculate_rns(output_tensor)

    # PSNR and SSIM
    psnr, ssim = calculate_psnr_ssim(input_img_pil, output_img_pil)output_img_pi

    # Save
    out_path = os.path.join(output_dir, os.path.basename(image_path))
    output_img_pil.save(out_path)
    output_img_pil.show(title="Denoised Image")

    # Display Results
    print("\n📊 Results (Image Quality Metrics):")
    print("---------------------------------------------------")
    print(f"✅ Saved denoised image: {out_path}")
    print(f"🔹 RNS (Input Image):   {rns_input:.6f}")
    print(f"🔹 RNS (Output Image):  {rns_output:.6f}")
    print(f"🔹 PSNR:                {psnr:.2f} dB")
    print(f"🔹 SSIM:                {ssim:.4f}")
    print("---------------------------------------------------")

# ----------------------------- #
# Folder and Video Logic (Optional)
# ----------------------------- #
def process_folder(folder_path):
    for fname in os.listdir(folder_path):
        if fname.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
            img_path = os.path.join(folder_path, fname)
            process_single_image(img_path)

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out_path = os.path.join(output_dir, "denoised_video.avi")
    out = None
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        img = Image.fromarray(gray)
        input_tensor, output_tensor = denoise_image(img)

        input_img_pil = to_pil(input_tensor)
        output_img_pil = to_pil(output_tensor)

        rns_input = calculate_rns(input_tensor)
        rns_output = calculate_rns(output_tensor)
        psnr, ssim = calculate_psnr_ssim(input_img_pil, output_img_pil)

        den_frame = np.array(output_img_pil)

        if out is None:
            h, w = den_frame.shape
            out = cv2.VideoWriter(out_path, fourcc, 20.0, (w, h), isColor=False)

        out.write(den_frame)

        print(f"🎞️ Frame {frame_idx:03d} | RNS_in: {rns_input:.4f} | RNS_out: {rns_output:.4f} | PSNR: {psnr:.2f} | SSIM: {ssim:.4f}")
        frame_idx += 1

    cap.release()
    if out:
        out.release()
        print(f"\n✅ Denoised video saved to: {out_path}")

# ----------------------------- #
# Main Entry
# ----------------------------- #
if os.path.isdir(input_path):
    print("📁 Detected: Directory of images")
    process_folder(input_path)
elif input_path.lower().endswith((".mp4", ".avi", ".mov", ".mkv", ".mpg", ".mpeg")):
    print("🎞️ Detected: Video file")
    process_video(input_path)
elif input_path.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
    print("🖼️ Detected: Single image")
    process_single_image(input_path)
elif input_path.lower() == "realtime":
    print("🟢 Realtime not supported with quality metrics.")
else:
    raise ValueError("❌ Unsupported input type.")


🖼️ Detected: Single image


TypeError: calculate_psnr_ssim() missing 1 required positional argument: 'img2'

In [29]:
import os
import cv2
import torch
import numpy as np
from torchvision import transforms as T
from PIL import Image
from model import LSC_CNN  # Ensure your model is defined correctly
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

# ----------------------------- #
# Configuration
# ----------------------------- #
input_path = "/home/himanshu/outputimage.jpg"  # Change as needed
output_dir = "/home/himanshu/Desktop/NUC_DATA/nuc_dataset/dataset/test_data"
model_path = "checkpoints/final_modelone.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(output_dir, exist_ok=True)

# ----------------------------- #
# Load Model
# ----------------------------- #
model = LSC_CNN().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# ----------------------------- #
# Transforms
# ----------------------------- #
transform = T.ToTensor()
to_pil = T.ToPILImage()

# ----------------------------- #
# Helper: RNS Calculation
# ----------------------------- #
def calculate_rns(tensor_img):
    mean_val = torch.mean(tensor_img)
    std_val = torch.std(tensor_img)
    return (std_val / (mean_val + 1e-8)).item()

# ----------------------------- #
# PSNR and SSIM Calculation
# ----------------------------- #
def calculate_psnr_ssim(img1, img2):
    img1_np = np.array(img1, dtype=np.float32) / 255.0
    img2_np = np.array(img2, dtype=np.float32) / 255.0
    psnr = compare_psnr(img1_np, data_range=1.0)
    ssim = compare_ssim(img1_np, img2_np, data_range=1.0)
    return psnr, ssim

# ----------------------------- #
# Denoising Inference
# ----------------------------- #
def denoise_image(img: Image.Image):
    img_gray = img.convert("L")
    img_tensor = transform(img_gray).unsqueeze(0).to(device)
    with torch.no_grad():
        output_tensor = model(img_tensor)
    input_tensor = img_tensor.squeeze().cpu().clamp(0, 1)
    output_tensor = output_tensor.squeeze().cpu().clamp(0, 1)
    return input_tensor, output_tensor

# ----------------------------- #
# Process Single Image
# ----------------------------- #
def process_single_image(image_path):
    img = Image.open(image_path).convert("L")
    input_tensor, output_tensor = denoise_image(img)

    # Convert to PIL for metric calculation
    input_img_pil = to_pil(input_tensor)
    output_img_pil = to_pil(output_tensor)

    # RNS
    rns_input = calculate_rns(input_tensor)
    rns_output = calculate_rns(output_tensor)

    # PSNR and SSIM
    psnr, ssim = calculate_psnr_ssim(input_img_pil, output_img_pil)output_img_pi

    # Save
    out_path = os.path.join(output_dir, os.path.basename(image_path))
    output_img_pil.save(out_path)
    output_img_pil.show(title="Denoised Image")

    # Display Results
    print("\n📊 Results (Image Quality Metrics):")
    print("---------------------------------------------------")
    print(f"✅ Saved denoised image: {out_path}")
    print(f"🔹 RNS (Input Image):   {rns_input:.6f}")
    print(f"🔹 RNS (Output Image):  {rns_output:.6f}")
    print(f"🔹 PSNR:                {psnr:.2f} dB")
    print(f"🔹 SSIM:                {ssim:.4f}")
    print("---------------------------------------------------")

# ----------------------------- #
# Folder and Video Logic (Optional)
# ----------------------------- #
def process_folder(folder_path):
    for fname in os.listdir(folder_path):
        if fname.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
            img_path = os.path.join(folder_path, fname)
            process_single_image(img_path)

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out_path = os.path.join(output_dir, "denoised_video.avi")
    out = None
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        img = Image.fromarray(gray)
        input_tensor, output_tensor = denoise_image(img)

        input_img_pil = to_pil(input_tensor)
        output_img_pil = to_pil(output_tensor)

        rns_input = calculate_rns(input_tensor)
        rns_output = calculate_rns(output_tensor)
        psnr, ssim = calculate_psnr_ssim(input_img_pil, output_img_pil)

        den_frame = np.array(output_img_pil)

        if out is None:
            h, w = den_frame.shape
            out = cv2.VideoWriter(out_path, fourcc, 20.0, (w, h), isColor=False)

        out.write(den_frame)

        print(f"🎞️ Frame {frame_idx:03d} | RNS_in: {rns_input:.4f} | RNS_out: {rns_output:.4f} | PSNR: {psnr:.2f} | SSIM: {ssim:.4f}")
        frame_idx += 1

    cap.release()
    if out:
        out.release()
        print(f"\n✅ Denoised video saved to: {out_path}")

# ----------------------------- #
# Main Entry
# ----------------------------- #
if os.path.isdir(input_path):
    print("📁 Detected: Directory of images")
    process_folder(input_path)
elif input_path.lower().endswith((".mp4", ".avi", ".mov", ".mkv", ".mpg", ".mpeg")):
    print("🎞️ Detected: Video file")
    process_video(input_path)
elif input_path.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
    print("🖼️ Detected: Single image")
    process_single_image(input_path)
elif input_path.lower() == "realtime":
    print("🟢 Realtime not supported with quality metrics.")
else:
    raise ValueError("❌ Unsupported input type.")


SyntaxError: invalid syntax (744489226.py, line 80)

In [30]:
import os
import cv2
import torch
import numpy as np
from torchvision import transforms as T
from PIL import Image
#from model import LSC_CNN  # Ensure your model is defined correctly
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

# ----------------------------- #
# Configuration
# ----------------------------- #
input_path = "/home/himanshu/outputimage.jpg"  # Change as needed
output_dir = "/home/himanshu/Desktop/NUC_DATA/nuc_dataset/dataset/test_data"
model_path = "checkpoints/final_modelone.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(output_dir, exist_ok=True)

# ----------------------------- #
# Load Model
# ----------------------------- #
model = LSC_CNN().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# ----------------------------- #
# Transforms
# ----------------------------- #
transform = T.ToTensor()
to_pil = T.ToPILImage()

# ----------------------------- #
# Helper: RNS Calculation
# ----------------------------- #
def calculate_rns(tensor_img):
    mean_val = torch.mean(tensor_img)
    std_val = torch.std(tensor_img)
    return (std_val / (mean_val + 1e-8)).item()

# ----------------------------- #
# PSNR and SSIM Calculation
# ----------------------------- #
def calculate_psnr_ssim(img1, img2):
    img1_np = np.array(img1, dtype=np.float32) / 255.0
    img2_np = np.array(img2, dtype=np.float32) / 255.0
    psnr = compare_psnr(img1_np, img2_np, data_range=1.0)
    ssim = compare_ssim(img1_np, img2_np, data_range=1.0)
    return psnr, ssim

# ----------------------------- #
# Denoising Inference
# ----------------------------- #
def denoise_image(img: Image.Image):
    img_gray = img.convert("L")
    img_tensor = transform(img_gray).unsqueeze(0).to(device)
    with torch.no_grad():
        output_tensor = model(img_tensor)
    input_tensor = img_tensor.squeeze().cpu().clamp(0, 1)
    output_tensor = output_tensor.squeeze().cpu().clamp(0, 1)
    return input_tensor, output_tensor

# ----------------------------- #
# Process Single Image
# ----------------------------- #
def process_single_image(image_path):
    img = Image.open(image_path).convert("L")
    input_tensor, output_tensor = denoise_image(img)

    # Convert to PIL for metric calculation
    input_img_pil = to_pil(input_tensor)
    output_img_pil = to_pil(output_tensor)

    # RNS
    rns_input = calculate_rns(input_tensor)
    rns_output = calculate_rns(output_tensor)

    # PSNR and SSIM
    psnr_input, ssim_input = calculate_psnr_ssim(input_img_pil, input_img_pil)
    psnr_output, ssim_output = calculate_psnr_ssim(input_img_pil, output_img_pil)

    # Save and Show Output Image
    out_path = os.path.join(output_dir, os.path.basename(image_path))
    output_img_pil.save(out_path)
    output_img_pil.show(title="Denoised Image")

    # Display Results in Table Format
    print("\n📊 Results (Image Quality Metrics):")
    print("------------------------------------------------------------")
    print(f"{'Metric':<15} | {'Input Image':<15} | {'Denoised Image':<15}")
    print("------------------------------------------------------------")
    print(f"{'RNS':<15} | {rns_input:<15.6f} | {rns_output:<15.6f}")
    print(f"{'PSNR (dB)':<15} | {psnr_input:<15.2f} | {psnr_output:<15.2f}")
    print(f"{'SSIM':<15} | {ssim_input:<15.4f} | {ssim_output:<15.4f}")
    print("------------------------------------------------------------")
    print(f"✅ Saved denoised image: {out_path}")

# ----------------------------- #
# Folder and Video Logic (Optional)
# ----------------------------- #
def process_folder(folder_path):
    for fname in os.listdir(folder_path):
        if fname.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
            img_path = os.path.join(folder_path, fname)
            process_single_image(img_path)

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out_path = os.path.join(output_dir, "denoised_video.avi")
    out = None
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        img = Image.fromarray(gray)
        input_tensor, output_tensor = denoise_image(img)

        input_img_pil = to_pil(input_tensor)
        output_img_pil = to_pil(output_tensor)

        rns_input = calculate_rns(input_tensor)
        rns_output = calculate_rns(output_tensor)
        psnr_output, ssim_output = calculate_psnr_ssim(input_img_pil, output_img_pil)

        den_frame = np.array(output_img_pil)

        if out is None:
            h, w = den_frame.shape
            out = cv2.VideoWriter(out_path, fourcc, 20.0, (w, h), isColor=False)

        out.write(den_frame)

        print(f"🎞️ Frame {frame_idx:03d} | RNS_in: {rns_input:.4f} | RNS_out: {rns_output:.4f} | PSNR: {psnr_output:.2f} | SSIM: {ssim_output:.4f}")
        frame_idx += 1

    cap.release()
    if out:
        out.release()
        print(f"\n✅ Denoised video saved to: {out_path}")

# ----------------------------- #
# Main Entry
# ----------------------------- #
if os.path.isdir(input_path):
    print("📁 Detected: Directory of images")
    process_folder(input_path)
elif input_path.lower().endswith((".mp4", ".avi", ".mov", ".mkv", ".mpg", ".mpeg")):
    print("🎞️ Detected: Video file")
    process_video(input_path)
elif input_path.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
    print("🖼️ Detected: Single image")
    process_single_image(input_path)
elif input_path.lower() == "realtime":
    print("🟢 Realtime not supported with quality metrics.")
else:
    raise ValueError("❌ Unsupported input type.")


🖼️ Detected: Single image

📊 Results (Image Quality Metrics):
------------------------------------------------------------
Metric          | Input Image     | Denoised Image 
------------------------------------------------------------
RNS             | 1.274183        | 1.337069       
PSNR (dB)       | inf             | 24.80          
SSIM            | 1.0000          | 0.3748         
------------------------------------------------------------
✅ Saved denoised image: /home/himanshu/Desktop/NUC_DATA/nuc_dataset/dataset/test_data/outputimage.jpg


/home/himanshu/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/pytorch/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/himanshu/.local/lib/python3.10/site-packages/skimage/metrics/simple_metrics.py:168: RuntimeWarning: divide by zero encountered in scalar divide
  return 10 * np.log10((data_range**2) / err)
